# Exercise 3. Part 2. Hyperparameter search

## Learning goals
* Practical experience in tuning hyperparameters of neural nets.

In [ ]:
skip_training = False  # Set this flag to True before validation and submission

In [ ]:
# During evaluation, this cell sets skip_training to True
# skip_training = True

In [ ]:
# Select data directory
import os
if os.path.isdir('/coursedata'):
    course_data_dir = '/coursedata'
elif os.path.isdir('../data'):
    course_data_dir = '../data'
else:
    # Specify course_data_dir on your machine
    # course_data_dir = ...
    # YOUR CODE HERE
    raise NotImplementedError()

print('The data directory is %s' % course_data_dir)

In [ ]:
import os
import itertools
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

In [ ]:
# Select device which you are going to use for training
device = torch.device("cpu")

In [ ]:
if skip_training:
    # The models are always evaluated on CPU
    device = torch.device("cpu")

## Grid search

Your first task is to implement grid search in the cell below. You are allowed to use only modules imported in the previos cell.

In [ ]:
def grid_search(*iterables):
    """
    Args:
      iterables: Each iterable is, e.g., a list (tuple or a numpy arrrays) containing grid values
                  for one of the tuned parameter.
    
    Returns:
      An iterator over all combinations of the grid values of the given iterables.
      Each object returned by the iterator is a tuple whose i-th element is one of the grid values from the
      i-th input iterable.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# Let's test your implementation
param1 = [0.1, 0.2, 0.3]  # Iterable with grid values of parameter 1
param2 = [0.4, 0.5]       # Iterable with grid values of parameter 2
param3 = [0.6, 0.7, 0.8]  # Iterable with grid values of parameter 3
for i in grid_search(param1, param2, param3):
    print(i)

## Random search

Your second task is to implement random search.

In [ ]:
def random_search(n, *param_ranges):
    """
    Args:
      n (int):      Number of hyperparameter combinations to be generated.
      param_ranges: Each of the given arguments must be a list [`low`, `high`] where low
                     defines the `lower` and `high` defines the upper boundaries of the sampling interval
                     for the corresponding parameter.
    Returns:
      An iterator over n combinations of the hyperparameters. Each hyperparameter value is drawn uniformly
      from interval [low, high] specified by the corresponding input argument.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
n = 10  # Number of hyperparameter combinations
param_range1 = [0.1, 0.9]  # lower and upper boundaries for parameter 1 
param_range2 = [1.1, 1.9]  # lower and upper boundaries for parameter 2
param_range3 = [2.1, 2.9]  # lower and upper boundaries for parameter 3
for i in random_search(n, param_range1, param_range2, param_range3):
    print(i)

## Hyperparameter search on a small dataset

Let us tune the hyperparameters of an MLP network to classify wines from the wine dataset.

In [ ]:
# Load the data
data_dir = os.path.join(course_data_dir, 'winequality')
print('Data loaded from %s' % data_dir)

df = pd.concat([
    pd.read_csv(os.path.join(data_dir, 'winequality-red.csv'), delimiter=';'),
    pd.read_csv(os.path.join(data_dir, 'winequality-white.csv'), delimiter=';')
])

x = df.loc[:, df.columns != 'quality'].values
y = df['quality'].values >= 7  # Convert to a binary classification problem

# Split into training, validation and test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=1, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=1, shuffle=True)

In [ ]:
# Scaling to zero mean and unit variance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

In [ ]:
# We will use an MLP with two hidden layers and dropout
n_inputs = 11

class MLP(nn.Module):
    def __init__(self, sizes, p=0):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(sizes[0], sizes[1]),
            nn.Dropout(p),
            nn.Tanh(),
            nn.Linear(sizes[1], sizes[2]),
            nn.Dropout(p),
            nn.Tanh(),
            nn.Linear(sizes[2], sizes[3])
        )
        
    def forward(self, x):
        return self.net(x)

In [ ]:
# Compute accuracy of a trained MLP on a given dataset
def compute_accuracy(x_test_scaled, y_test, mlp):
    mlp.eval()
    x = torch.tensor(x_test_scaled, dtype=torch.float, device=device)
    outputs = mlp.forward(x)
    logits = outputs.cpu().data.numpy()
    pred_test = logits.argmax(axis=1)
    test_accuracy = accuracy_score(pred_test, y_test)
    return test_accuracy

In [ ]:
# Training procedure
def train(x_train_scaled, y_train, mlp, lrate, print_every):
    optimizer = torch.optim.Adam(mlp.parameters(), lr=lrate)
    scheduler = StepLR(optimizer, step_size=20, gamma=0.95)
    
    n_epochs = 1000

    train_accuracy_history = []
    val_accuracy_history = []

    for epoch in range(n_epochs):
        mlp.train()
        scheduler.step()
        x = torch.tensor(x_train_scaled, device=device, dtype=torch.float)
        y = torch.tensor(y_train.astype(int), device=device).long()

        optimizer.zero_grad()
        outputs = mlp.forward(x)
        loss = F.cross_entropy(outputs, y)
        loss.backward()
        optimizer.step()

        if (epoch % print_every) == 0:
            # Store the progress of training
            with torch.no_grad():
                logits = outputs.cpu().data.numpy()
                pred_train = logits.argmax(axis=1)
                train_accuracy = accuracy_score(pred_train, y_train)
                train_accuracy_history.append(train_accuracy)
                
                # Compute validation accuracy
                val_accuracy = compute_accuracy(x_val_scaled, y_val, mlp)
                val_accuracy_history.append(val_accuracy)
                print('Train Epoch {}: Loss: {:.6f} Train accuracy {:.2f} Valdation accuracy {:.2f}'.format(
                    epoch, loss.item(), train_accuracy, val_accuracy))
    
    return mlp, train_accuracy_history, val_accuracy_history

Let us tune the hyperparameters using our own implementation of random search. Try at least 10 parameter combinations.

In [ ]:
n = 10  # Number of parameter combinations

n_hidden1_range = [10, 400]
h_hidden2_range = [10, 400]
log_lrate_range = [np.log(0.001), np.log(0.1)]
log_dropout_range = [np.log(0.001), np.log(0.3)]

hyperparameters = []
accuracies = []
if not skip_training:
    for (n_hidden1, n_hidden2, log_lrate, log_dropout) in \
            random_search(n, n_hidden1_range, h_hidden2_range, log_lrate_range, log_dropout_range):
        n_hidden1, n_hidden2 = int(n_hidden1), int(n_hidden2)
        lrate, dropout = np.exp(log_lrate), np.exp(log_dropout)
        hyperparameters.append([n_hidden1, n_hidden2, lrate, dropout])
        print('Hyperparameters: ', hyperparameters[-1])
        mlp = MLP([n_inputs, n_hidden1, n_hidden2, 2], p=dropout)
        print(mlp)
        mlp.to(device)
        mlp, train_accuracy_history, val_accuracy_history = train(x_train_scaled, y_train, mlp, lrate, print_every=199)
        accuracies.append(val_accuracy_history[-1])
        print('Final accuracy:', accuracies[-1])
        #print(compute_accuracy(x_test_scaled, y_test, mlp))

In [ ]:
hyperparameters = np.array(hyperparameters)
accuracies = np.array(accuracies)

In [ ]:
# Save results to disk. Submit file `3_random_search.npz` together with your notebook.
hs_filename = '3_random_search.npz'
if not skip_training:
    try:
        do_save = input('Do you want to save the results of hyperparameter search (type yes to confirm)? ').lower()
        if do_save == 'yes':
            np.savez(hs_filename,
                     hyperparameters=hyperparameters,
                     accuracies=accuracies)
            print('Results saved to %s' % hs_filename)
        else:
            print('Results not saved')
    except:
        raise Exception('The notebook should be run or validated with skip_training=True.')
else:
    rs = np.load(hs_filename)
    hyperparameters = rs['hyperparameters']
    accuracies = rs['accuracies']
    print('Results loaded from %s' % hs_filename)

In [ ]:
# Print results
print('#hidden1 #hidden2 lrate dropout accuracy')
ix = accuracies.argsort()[-1::-1]
for (n_hidden1, n_hidden2, lrate, dropout), accuracy in zip(hyperparameters[ix], accuracies[ix]):
    print('%8d %8d %5.3f %7.3f %8.3f' % (n_hidden1, n_hidden2, lrate, dropout, accuracy))

## Train the network with the best hyperparameters including validation data

In [ ]:
# Select hyperparameters producing the best validation accuracy
best_run = accuracies.argmax()
n_hidden1, n_hidden2, lrate, dropout = hyperparameters[best_run]
sizes = [n_inputs, int(n_hidden1), int(n_hidden2), 2]
mlp = MLP(sizes, p=dropout)
mlp.to(device)
print('Best architecture:', mlp)
print('Best validataion accuracy: %.3f' % accuracies[best_run])

In [ ]:
# Train the network with the best hyperparameters using also validation data
if not skip_training:
    mlp, train_accuracy_history, val_accuracy_history = train(
        np.vstack((x_train_scaled, x_val_scaled)), np.hstack((y_train, y_val)),
        mlp, lrate, print_every=199
    )

In [ ]:
# Save the network to a file, submit this file together with your notebook
filename = '3_mlp.pth'
if not skip_training:
    try:
        do_save = input('Do you want to save the model? ').lower()
        if do_save == 'yes':
            torch.save(mlp.state_dict(), filename)
            print('Model saved to %s' % filename)
        else:
            print('Model not saved')
    except:
        raise Exception('The notebook should be run or validated with skip_training=False.')
else:
    mlp.load_state_dict(torch.load(filename, map_location=lambda storage, loc: storage))
    mlp.to(device)
    print('Model loaded from %s' % filename)

In [ ]:
# Test the accuracy of the network trained with the best hyperparameters
mlp.eval()
test_accuracy = compute_accuracy(x_test_scaled, y_test, mlp)
print("Test accuracy of the best model: %.3f" % test_accuracy)

The accuracy should be greater than 0.85.